In [ ]:
#export
from fastai2.basics import *
from fastai2.tabular.core import *
from fastai2.tabular.model import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
#default_exp tabular.learner

# Tabular learner

> The function to immediately get a `Learner` ready to train for tabular data

## Main functions

In [ ]:
#export
class TabularLearner(Learner):
    "`Learner` for tabular data"
    def predict(self, row):
        tst_to = self.dbunch.valid_ds.new(pd.DataFrame(row).T)
        tst_to.process()
        dl = self.dbunch.valid_dl.new(tst_to)
        inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
        i = getattr(self.dbunch, 'n_inp', -1)
        b = (*tuplify(inp),*tuplify(dec_preds))
        full_dec = self.dbunch.decode((*tuplify(inp),*tuplify(dec_preds)))
        return full_dec,dec_preds[0],preds[0]

In [ ]:
#export
@delegates(Learner.__init__)
def tabular_learner(dbunch, layers=None, emb_szs=None, config=None, **kwargs):
    "Get a `Learner` using `data`, with `metrics`, including a `TabularModel` created using the remaining params."
    if config is None: config = tabular_config()
    if layers is None: layers = [200,100]
    to = dbunch.train_ds
    emb_szs = get_emb_sz(dbunch.train_ds, {} if emb_szs is None else emb_szs)
    model = TabularModel(emb_szs, len(dbunch.cont_names), get_c(dbunch), layers, **config)
    return TabularLearner(dbunch, model, **kwargs)

In [ ]:
#export
@typedispatch
def show_results(x:Tabular, y:Tabular, samples, outs, ctxs=None, max_n=10, **kwargs):
    df = x.all_cols[:max_n]
    for n in x.y_names: df[n+'_pred'] = y[n][:max_n].values
    display_df(df)

## Integration example with training

In [ ]:
from fastai2.callback.all import *

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]
splits = RandomSplitter()(range_of(df))

In [ ]:
to = TabularPandas(df, procs, cat_names, cont_names, y_names="salary", splits=splits)

In [ ]:
dbunch = to.databunch(bs=64)
dbunch.show_batch()

,workclass,education,marital-status,occupation,relationship,race,age_na,fnlwgt_na,education-num_na,age,fnlwgt,education-num,salary
0,Private,7th-8th,Married-civ-spouse,Sales,Husband,White,False,False,False,63.000001,30269.995641,4.0,<50k
1,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,False,False,False,17.000001,276718.001588,9.0,<50k
2,Local-gov,HS-grad,Never-married,Handlers-cleaners,Other-relative,Black,False,False,False,25.000000,348986.004148,9.0,<50k
3,Private,Some-college,Married-civ-spouse,Handlers-cleaners,Husband,Black,False,False,False,21.000000,202373.000381,10.0,<50k
4,Private,Some-college,Never-married,Craft-repair,Not-in-family,White,False,False,False,32.000000,207937.000226,10.0,<50k
5,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,45.000000,337824.996847,9.0,<50k
6,Private,7th-8th,Married-civ-spouse,Machine-op-inspct,Wife,White,False,False,False,38.000000,100294.997626,4.0,>=50k
7,Self-emp-inc,Assoc-voc,Married-civ-spouse,Craft-repair,Husband,White,False,False,False,39.000000,143122.999187,11.0,>=50k
8,Private,12th,Married-civ-spouse,Machine-op-inspct,Husband,White,False,False,False,49.000000,105444.002488,8.0,<50k
9,Local-gov,Doctorate,Married-civ-spouse,Prof-specialty,Wife,Black,False,False,False,51.000000,166460.999877,16.0,>=50k


In [ ]:
learn = tabular_learner(dbunch, [200,100], metrics=accuracy)

In [ ]:
learn.dbunch.train_dl.device

device(type='cuda', index=0)

In [ ]:
to_device

<function fastai2.torch_core.to_device(b, device=None)>

In [ ]:
to.to(default_device())

AttributeError: to

In [ ]:
learn.fit_one_cycle(1)

RuntimeError: Expected object of device type cuda but got device type cpu for argument #3 'index' in call to _th_index_select

In [ ]:
%debug

> /home/sgugger/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py(1484)embedding()
   1482         # remove once script supports set_grad_enabled
   1483         _no_grad_embedding_renorm_(weight, input, max_norm, norm_type)
-> 1484     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   1485 
   1486 

ipdb> u
> /home/sgugger/anaconda3/lib/python3.7/site-packages/torch/nn/modules/sparse.py(114)forward()
    112         return F.embedding(
    113             input, self.weight, self.padding_idx, self.max_norm,
--> 114             self.norm_type, self.scale_grad_by_freq, self.sparse)
    115 
    116     def extra_repr(self):

ipdb> input,device
*** NameError: name 'device' is not defined
ipdb> p input.device
device(type='cpu')
ipdb> p self.weight.device
device(type='cuda', index=0)
ipdb> q


In [ ]:
learn.show_results()

In [ ]:
learn.predict(df.iloc[0])

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()